In [1]:
import os
import cv2
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

In [21]:
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from keras.models import model_from_json

### Read Images

In [4]:
FILE_PATH = 'data/dataset_B_Eye_Images'
IMG_SIZE = 24

def read_images():
    images = []
    labels = []
    for file_name in os.listdir(FILE_PATH):
        file_path = FILE_PATH + '/' + file_name
        for img_name in os.listdir(file_path):
            if not img_name.startswith('.'):
                if img_name.endswith('.jpg'):
                    img = cv2.imread(file_path + '/' + img_name)
                    image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                    new_img = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                    images.append(new_img)
                    
                    if file_name == 'closedLeftEyes' or file_name == 'closedRightEyes':
                        label = 0
                    else:
                        label = 1
                    labels.append(label)
    return np.array(images), np.array(labels)

### Save model:

In [5]:
def save_model(model):
    model_json = model.to_json()
    with open("models/eye_status_model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("models/eye_status_model.h5")
    print("Model saved")

### Train model:

In [6]:
def train(X_train, y_train):
    model = Sequential()

    model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
    model.add(AveragePooling2D())

    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
    model.add(AveragePooling2D())

    model.add(Flatten())

    model.add(Dense(units=120, activation='relu'))

    model.add(Dense(units=84, activation='relu'))

    model.add(Dense(units=1, activation = 'sigmoid'))


    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(X_train,y_train,batch_size=50,epochs=20,verbose=1)
    
    return model


### read images:

In [7]:
full_data_x, full_data_y = read_images()

In [8]:
print("dataset size : ", full_data_x.shape, full_data_y.shape)

dataset size :  (4846, 24, 24) (4846,)


In [9]:
print("One Image size: ", full_data_x[2].shape)

One Image size:  (24, 24)


In [10]:
train_x, test_x, train_y, test_y = train_test_split(full_data_x, full_data_y, test_size=0.2)

### save data

In [11]:
np.save('data/train_imgs.npy', train_x)
np.save('data/train_y.npy', train_y)
np.save('data/test_imgs.npy', test_x)
np.save('data/test_y.npy', test_y)

### load data

In [12]:
train_x = np.load('data/train_imgs.npy')
train_y = np.load('data/train_y.npy')
test_x = np.load('data/test_imgs.npy')
test_y = np.load('data/test_y.npy')

In [13]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((3876, 24, 24), (3876,), (970, 24, 24), (970,))

In [14]:
X_train = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], train_x.shape[2], 1))
X_test = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], test_x.shape[2], 1))
y_train = train_y
y_test = test_y

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3876, 24, 24, 1), (970, 24, 24, 1), (3876,), (970,))

### Train model:

In [20]:
model = train(X_train, y_train)

Epoch 1/20
3876/3876 [==============================] - 1s 188us/step - loss: 1.1744 - accuracy: 0.7288
Epoch 2/20
3876/3876 [==============================] - 1s 169us/step - loss: 0.2616 - accuracy: 0.9038
Epoch 3/20
3876/3876 [==============================] - 1s 162us/step - loss: 0.2166 - accuracy: 0.9118
Epoch 4/20
3876/3876 [==============================] - 1s 150us/step - loss: 0.1550 - accuracy: 0.9383
Epoch 5/20
3876/3876 [==============================] - 1s 149us/step - loss: 0.1435 - accuracy: 0.9438
Epoch 6/20
3876/3876 [==============================] - 1s 144us/step - loss: 0.1370 - accuracy: 0.9458
Epoch 7/20
3876/3876 [==============================] - 1s 144us/step - loss: 0.1284 - accuracy: 0.9507
Epoch 8/20
3876/3876 [==============================] - 1s 165us/step - loss: 0.1065 - accuracy: 0.9587
Epoch 9/20
3876/3876 [==============================] - 1s 166us/step - loss: 0.0975 - accuracy: 0.9613
Epoch 10/20
3876/3876 [==============================] - 1s 156u

In [17]:
save_model(model)

Model saved


### Testing

In [18]:
predictions = model.evaluate(X_test,y_test)

970/970 [==============================] - 0s 155us/step


In [19]:
print("Accurecy ", predictions[1]*100)

Accurecy  94.02061700820923


### Predict

In [27]:
img = cv2.imread('data/dataset_B_Eye_Images/openLeftEyes/Alejandro_Gonzalez_Inarritu_0001_L.jpg')
image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
new_img = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
imag = np.reshape(np.array(new_img), (1, IMG_SIZE, IMG_SIZE, 1))
prediction = model.predict(imag)
print(prediction)

[[0.99980146]]
